<a href="https://colab.research.google.com/github/marosimosi/nerf/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [7]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import os
import math

# Device

In [2]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device:', device)

Device: cuda


# Load Dataset

In [3]:
if not os.path.exists('tiny_nerf_data.npz'):
  !wget http://cseweb.ucsd.edu/~viscomp/projects/LF/papers/ECCV20/nerf/tiny_nerf_data.npz

dataset = np.load('tiny_nerf_data.npz')

images = dataset['images']
poses = dataset['poses']
focal = dataset['focal']

--2024-11-14 23:28:43--  http://cseweb.ucsd.edu/~viscomp/projects/LF/papers/ECCV20/nerf/tiny_nerf_data.npz
Resolving cseweb.ucsd.edu (cseweb.ucsd.edu)... 132.239.8.30
Connecting to cseweb.ucsd.edu (cseweb.ucsd.edu)|132.239.8.30|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cseweb.ucsd.edu//~viscomp/projects/LF/papers/ECCV20/nerf/tiny_nerf_data.npz [following]
--2024-11-14 23:28:43--  https://cseweb.ucsd.edu//~viscomp/projects/LF/papers/ECCV20/nerf/tiny_nerf_data.npz
Connecting to cseweb.ucsd.edu (cseweb.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12727482 (12M)
Saving to: ‘tiny_nerf_data.npz’

tiny_nerf_data.npz  100%[===================>]  12.14M  36.9MB/s    in 0.3s    

2024-11-14 23:28:43 (36.9 MB/s) - ‘tiny_nerf_data.npz’ saved [12727482/12727482]



In [4]:
print(f'Images shape: {images.shape}')
print(f'Poses shape: {poses.shape}')
print(f'Focal length: {focal}')

Images shape: (106, 100, 100, 3)
Poses shape: (106, 4, 4)
Focal length: 138.88887889922103


Το dataset περιέχει 106 εικόνες, με διαστάσεις που αντιστοιχούν στο πλάτος, το ύψος και το RGB.

Κάθε pose είναι ένας 4x4 transformation matrix. Αποτελείται από έναν 3x3 rotation matrix, έναν 3x1 translation vector στην τέταρτη στήλη, και την τελευταία γραμμή [0,0,0,1] για ομογενείς συντεταγμένες.

In [10]:
translations = poses[:, :3, 3]
distances = np.linalg.norm(translations, axis=1) #distances from [0,0,0]

# Set near and far based on min and max distances, with a margin
near = math.floor( max(0.1, distances.min() - 1) )
far = math.ceil( distances.max() + 1 )

print(f"Calculated near plane: {near}")
print(f"Calculated far plane: {far}")

Calculated near plane: 3
Calculated far plane: 6
